# Woman to Man Project

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import glob
import random
import os

from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import sys
import torch.nn as nn
import torch.nn.functional as F
import torch

import os
import numpy as np
import math
import itertools
import datetime
import time

from torchvision.utils import save_image, make_grid
from torchvision import datasets
from torch.autograd import Variable

**Code 1. Dataset**

Image to Image Translation (style transfer 일종)
 
 -cycle GAN 구조 -> 진짜 이미지를 통해 가짜 이미지를 생성한 후 계속해서 비교하면서 진짜 이미지와 가짜 이미지를 판별하면서 반복한다.

가짜 이미지 B^과 A^그리고 판별할 Da, Db가 필요하다.


In [ ]:
cd gdrive/MyDrive/python/data/'Dataset'

/content/gdrive/MyDrive/python/data/Dataset


In [ ]:
pip install -r requirements.txt

Processing /dlib-19.18.0-cp37-cp37m-linux_x86_64.whl
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/dlib-19.18.0-cp37-cp37m-linux_x86_64.whl'



In [ ]:
def to_rgb(image):
    rgb_image = Image.new("RGB", image.size)
    rgb_image.paste(image)
    return rgb_image
    #PIL의 Image모듈을 통해 흑백이미지를 RGB로 반환합니다.

In [ ]:
# # Python Image Library (pillow모듈)
#  주요기능에는 픽셀단위 조작/ 흐림, 윤곽보정 등 이미지 필터/ 선명하게, 밝기보정, 명암보정, 색보정 등 회상조정/ 이미지에 텍스트 추가 등
#  설치는 pip install pillow
#  from PIL import Image

In [ ]:

class ImageDataset(Dataset):
    def __init__(self, root, transforms_= None, unaligned=False, mode="train"):
        self.transform = transforms.Compose(transforms_)
        self.unaligned = unaligned
        # train 모드일 때는 trainA, trainB에 있는 디렉토리에서 이미지를 불러옵니다. 
        # unaligned = False는 학습할쌍을 고정시키지 않겠다는 뜻입니다.(무작위)
        # transform은 PIL이미지를 토치의 tensor자료형으로 바꿉니다.
    
            # glob 함수로 trainA 디렉토리의 이미지의 목록을 불러옵니다. 
        self.files_A = sorted(glob.glob(os.path.join(root, '%sA' % mode) + '/*.*')) #'/*,*'은 모든 파일을 뜻합니다. *.jpg로 파일 명을 설정할 수도 있다.
        self.files_B = sorted(glob.glob(os.path.join(root, '%sB' % mode) + '/*.*'))
        # if mode ="train": 으로 트레인 폴더의 A,B와 테스트 폴더의 A,B를 나누어도 되지만 같은 폴더에 넣는 것으로 '%sA'로 코드 간결화
    def __getitem__(self, index):
        # index값으로 이미지의목록 중 이미지 하나를 불러옵니다. 
        image_A = Image.open(self.files_A[index % len(self.files_A)])
        # unaligned 변수로 학습할 Pair를 랜덤으로 고릅니다.  
        if self.unaligned:
            image_B = Image.open(self.files_B[random.randint(0, len(self.files_B) - 1)])
        else:
            image_B = Image.open(self.files_B[index % len(self.files_B)])
       
        # Convert grayscale images to rgb
        if image_A.mode != "RGB":
            image_A = to_rgb(image_A)
        if image_B.mode != "RGB":
            image_B = to_rgb(image_B)
        item_A = self. transform(image_A)
        item_B = self. transform(image_B)
        # transform을 sorted앞에 붙여서 간결화 가능? 복잡해서 이게 더 깔끔?

        return {'A': item_A, 'B':item_B}


    def __len__(self):
        return max(len(self.files_A), len(self.files_B))

        # 앞으로 loader을 이용하여 배치사이즈 만큼 이미지를 불러올 것입니다.

**Code 2. Generator & Discriminator**

In [ ]:
# [가중치 초기화 함수]
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
        if hasattr(m, "bias") and m.bias is not None:
            torch.nn.init.constant_(m.bias.data, 0.0)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)
        # 여기 함수는 layer의 종류에 따라 가중치를 초기화 합니다.

In [ ]:
# Gnerator 구현을 위해 내부에 들어갈 la yor인 Residual Block을 구현합니다.
# Residual Block은 이전 layer와 현재 layer의 출력값을 더해서 foward 시키는 것을 기울기 소실 문제를 해결합니다.(더하기 연산으로는 작아지지 않으니)
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()

        self.block = nn.Sequential(
            nn.ReflectionPad2d(1), #점대칭방식, 가장 가까운 픽셀로 값복사(자연스런 이미지 생성) vs zero padding은 값을 지정합니다.
            nn.Conv2d(in_features, in_features, 3), 
            nn.InstanceNorm2d(in_features), #데이터 개별로 정규화를 진행합니다.
            # 정규화는 데이터 값 범위 조정을 하는 것으로 배치정규화는 배치단위로 진행합니다. instance 정규화는 이미지에 특화된 정규화 과정으로 이미지 개별에 정규화를 진행합니다.
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features),
        )

    def forward(self, x):
        return x + self.block(x) #입력 feature개수와 출력 feature개수를 일치시켜야 합니다.
        # Residual Block이 늘어날수록 더 많은 계층적인 정보가 담겨 더욱 그럴듯한 이미지가 생성됩니다.

In [ ]:
class GeneratorResNet(nn.Module):
    def __init__(self, input_shape, num_residual_blocks):
        super(GeneratorResNet, self).__init__()

        channels = input_shape[0]

        # Initial convolution block(초기의 convolution block 설정)
        out_features = 64
        model = [
            nn.ReflectionPad2d(channels),
            nn.Conv2d(channels, out_features, 7),
            nn.InstanceNorm2d(out_features),
            nn.ReLU(inplace=True),
        ]
        in_features = out_features

        # Downsampling을 2번하는 경우로 stride가 2로 이미지 크기를 반씩 줄입니다.
        for _ in range(2):
            out_features *= 2
            model += [
                nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True),
            ]
            in_features = out_features

        # Residual blocks
        for _ in range(num_residual_blocks):
            model += [ResidualBlock(out_features)]
            # num_risidual_blocks 만큼 residual block을 만듭니다. (입력 출력 일치)

        # Upsampling을 2번하면서 2배씩 크기를 늘립니다.
        for _ in range(2):
            out_features //= 2
            model += [
                nn.Upsample(scale_factor=2),
                nn.Conv2d(in_features, out_features, 3, stride=1, padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True),
            ]
            in_features = out_features

        # Output layer을 선언합니다. /입력 이미지 크기 = 출력 이미지 크기 / 마지막 층 활성화 함수는 nn.Tanh()입니다.
        model += [nn.ReflectionPad2d(channels), nn.Conv2d(out_features, channels, 7), nn.Tanh()]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, input_shape):
        super(Discriminator, self).__init__()

        channels, height, width = input_shape

        # Calculate output shape of image discriminator (PatchGAN) 
        # discriminator의 출력 크기를 정의 합니다.
        self.output_shape = (1, height // 2 ** 4, width // 2 ** 4)

        def discriminator_block(in_filters, out_filters, normalize=True):
            """Returns downsampling layers of each discriminator block"""
            # block은 stride 2로 점점 downsampling하며 출력 이미지 크기를 줄입니다.
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
            if normalize:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers
            
        # block 4번을 통과합니다. 그래서 16x16이 됩니다. (한번 통과시 반이 줄어듦) 
        self.model = nn.Sequential(
            *discriminator_block(channels, 64, normalize=False),
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(512, 1, 4, padding=1)
        )


    def forward(self, img):
        return self.model(img)

#  patch GAN의 Discriminator는 출력이 0이나 1의 값이 아닌 입력 이미지의 1/16인 이진화된 feature map이 나옵니다. 
  # if 이미지 크기가 256x256이면 patch GAN의 Discriminator의 출력 16x16으로 나옵니다.

What is a feature map in CNN?


The feature maps of a CNN capture the result of applying the filters to an input image. I.e at each layer, the feature map is the output of that layer. The reason for visualising a feature map for a specific input image is to try to gain some understanding of what features our CNN detects


**Code 3. Training**

In [ ]:
class opt :
 pretrained_model_path = "/content/gdrive/MyDrive/python/data/Dataset/saved_models/wom2man"
 # epoch to start training from 
 epoch_start = 130
 dataset_name="wom2man" #여기서 폴더가 지정되어 있어서 dataset안에 oldtonew폴더를 생성해야한다.
 channels = 3  #흑백의 경우 1 /RGB이미지는 3
 img_height = 256 
 img_width = 256 #가로세로크기 / 크기를 키울경우 cuda 메모리 오류가 생긴다.
 n_residual_blocks=9 #Generator에서의 Residual Block의 개수
 lr=0.0002 #learning rate
 b1=0.5 
 b2=0.999  #b1과 b2는 Adam opimizer에 대한 Hyper Parameter
 n_epochs=400 #목표200/ 했다가 비슷한 이미지가 생성되는 것을 확인하여 다른 방법이 필요하다고 느낌 or 에폭을 더 늘려보기
 init_epoch=0
 decay_epoch=120 #목표 100 /학습속도를 조절 처음에는 크게 그담에는 작은 단위로
 lambda_cyc=10.0
 lambda_id=5.0 #cycle-consistency loss 와 identity loss에 대한 람다값(id가 클수록 본래 색감 유지)
 n_cpu=8
 batch_size=1
 sample_interval=300
 checkpoint_interval= 10

In [ ]:

pretrained_model_path = "/content/gdrive/MyDrive/python/data/Dataset/saved_models/wom2man"
# epoch to start training from 
epoch_start = 130
dataset_name="wom2man" #여기서 폴더가 지정되어 있어서 dataset안에 oldtonew폴더를 생성해야한다.
channels = 3  #흑백의 경우 1 /RGB이미지는 3
img_height = 256 
img_width = 256 #가로세로크기 / 크기를 키울경우 cuda 메모리 오류가 생긴다.
n_residual_blocks=9 #Generator에서의 Residual Block의 개수
lr=0.0002 #learning rate
b1=0.5 
b2=0.999  #b1과 b2는 Adam opimizer에 대한 Hyper Parameter
n_epochs=400 #목표200/ 했다가 비슷한 이미지가 생성되는 것을 확인하여 다른 방법이 필요하다고 느낌 or 에폭을 더 늘려보기
init_epoch=0
decay_epoch=120 #목표 100 /학습속도를 조절 처음에는 크게 그담에는 작은 단위로
lambda_cyc=10.0
lambda_id=5.0 #cycle-consistency loss 와 identity loss에 대한 람다값(id가 클수록 본래 색감 유지)
n_cpu=8
batch_size=1
sample_interval=300
checkpoint_interval= 10

In [ ]:
# Create sample and checkpoint directories 
#샘플 이미지와 모델 가중치를 저장할 폴더를 생성.
os.makedirs("images/%s" % dataset_name, exist_ok=True)
os.makedirs("saved_models/%s" % dataset_name, exist_ok=True)
#exist_ok=True면 같은 폴더가 있어도 오류가 나지 않는다.

In [ ]:
# Losses(손실함수 정의)
criterion_GAN = torch.nn.MSELoss()
criterion_cycle = torch.nn.L1Loss()
criterion_identity = torch.nn.L1Loss()

In [ ]:
#모델 객체 선언하기
input_shape = (channels, img_height, img_width)

# Initialize generator and discriminator
G_AB = GeneratorResNet(input_shape, n_residual_blocks)
G_BA = GeneratorResNet(input_shape, n_residual_blocks)
D_A = Discriminator(input_shape)
D_B = Discriminator(input_shape)
# 스타일 A ->B :Generator G.AB
# 스타일 B ->A :Generator G.BA
# 스타일 A,B가 진짜인지 가짜인지 판별할 네트워크 D.a, D.b선언

In [ ]:
# GPU에 로드하기
cuda = torch.cuda.is_available()

if cuda:
    G_AB = G_AB.cuda()
    G_BA = G_BA.cuda()
    D_A = D_A.cuda()
    D_B = D_B.cuda()
    criterion_GAN.cuda()
    criterion_cycle.cuda()
    criterion_identity.cuda()

In [ ]:
if epoch_start != 0:
    # Load pretrained models 
    G_AB.load_state_dict(torch.load(f"{pretrained_model_path}/G_AB_130.pth")) #런타임 오류로 끊길 우려가 있어서 중간에 가중치를 로드받아서 진행하는 모델 생성
    G_BA.load_state_dict(torch.load(f"{pretrained_model_path}/G_BA_130.pth"))
    D_A.load_state_dict(torch.load(f"{pretrained_model_path}/D_A_130.pth"))
    D_B.load_state_dict(torch.load(f"{pretrained_model_path}/D_B_130.pth"))
  


# Initialize weights
else : 
    G_AB.apply(weights_init_normal)    # old=>new generator 가중치 초기화
    G_BA.apply(weights_init_normal)    # new=>old generator 가중치 초기화
    D_A.apply(weights_init_normal)    # old discriminator 가중치 초기화
    D_B.apply(weights_init_normal)    # new discriminator 가중치 초기화

In [ ]:
# Optimizers 정의
optimizer_G = torch.optim.Adam(
    itertools.chain(G_AB.parameters(), G_BA.parameters()), lr=lr, betas=(b1, b2)
)
optimizer_D_A = torch.optim.Adam(D_A.parameters(), lr=lr, betas=(b1, b2))
optimizer_D_B = torch.optim.Adam(D_B.parameters(), lr=lr, betas=(b1, b2))
# 앞서 정의한 hyper parameter lr,b1,b2로 optimizer를 정의합니다. 
# intertools,chain을 이용하면 optimizer가 여러 모델의 parameter를 하나의 모델을 다루는 것처럼 동작합니다.

In [ ]:
# 학습스케줄러
class LambdaLR:
    def __init__(self, n_epochs, offset, decay_start_epoch):
        assert (n_epochs - decay_start_epoch) > 0, "Decay must start before the training session ends!"
        self.n_epochs = n_epochs
        self.offset = offset
        self.decay_start_epoch = decay_start_epoch

    def step(self, epoch):
        return 1.0 - max(0, epoch + self.offset - self.decay_start_epoch) / (self.n_epochs - self.decay_start_epoch)
        # 람다LR클래스를 정의합니다. lr를 decay할 epoch을 정할 수 있습니다.
        

In [ ]:
# Learning rate update schedulers
lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(
    optimizer_G, lr_lambda=LambdaLR(n_epochs, init_epoch, decay_epoch).step
)
lr_scheduler_D_A = torch.optim.lr_scheduler.LambdaLR(
    optimizer_D_A, lr_lambda=LambdaLR(n_epochs, init_epoch, decay_epoch).step
)
lr_scheduler_D_B = torch.optim.lr_scheduler.LambdaLR(
    optimizer_D_B, lr_lambda=LambdaLR(n_epochs, init_epoch, decay_epoch).step
)

In [ ]:
# 텐서 연산에 사용할 Tensor 자료형을 정의합니다.
Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor

In [ ]:
# Torch변수가 requires_grad=True로 지정되어 있다면 매 연산마다 Gradient를 저장하므로 
# cycle GAN학습을 위해서는 ReplayBuffer클래스를 통해 이미지를 따로 저장해야 합니다.
class ReplayBuffer:
    def __init__(self, max_size=50):
        assert max_size > 0, "Empty buffer or trying to create a black hole. Be careful."
        self.max_size = max_size
        self.data = []

    def push_and_pop(self, data):
        to_return = []
        for element in data.data:
            element = torch.unsqueeze(element, 0)
            if len(self.data) < self.max_size:
                self.data.append(element)
                to_return.append(element)
            else:
                if random.uniform(0, 1) > 0.5:
                    i = random.randint(0, self.max_size - 1)
                    to_return.append(self.data[i].clone())
                    self.data[i] = element
                else:
                    to_return.append(element)
        return Variable(torch.cat(to_return))

In [ ]:
# Buffers of previously generated samples
fake_A_buffer = ReplayBuffer()
fake_B_buffer = ReplayBuffer()

In [ ]:
# Dataset클래스로 이미지를 불러오는 방식에 대하여 정의합니다.
# Image transformations
transforms_ = [
    transforms.Resize(int(img_height * 1.12), Image.BICUBIC), #PIL이미지를 가로, 세로로 1.12배 확대합니다. 이때 보간 방식은 BICUBIC을 사용합니다.
    transforms.RandomCrop((img_height, img_width)), #PIL이미지를 가로, 세로 길이만큼 무작위로 잘라냅니다.
    transforms.RandomHorizontalFlip(),#PIL이미지를 무작위로 좌우로 뒤집습니다,
    transforms.ToTensor(), #PIL이미지를 0~1사이의 Tensor자료형으로 변환합니다.
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), #RGB 채널별로 픽셀값이 평균 0.5, 표준편차가 0.5가 되도록 정규화합니다.
]

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


학습 혹은 평가 중 이미지 데이터를 불러올 Dataloader를 정의합니다. 앞서 정의한 image dataset클래스로 selfie2anime데이터 폴더로부터 transform_를 적용한 이미지를 배치 사이즈만큼 불러옵니다. 또한 num_workers로 cpu유틸리티를 설정할 수 있습니다. shuffle변수를 True 혹은 False로 설정해 이미지를 무작위로 혹은 순차적으로 불러올 수 있습니다. colab에서 실습할 경우 GPU 메모리 부족으로 OOM이슈가 나올 수 있으므로 배치사이즈를 줄여 실행 하면 됩니다.

OutOfMemory Error
OOM의 원인은 다양합니다. 반드시 Memory Leak이 아닐 수도 있습니다. 처리 메모리 부족?

In [ ]:
# Training data loader
dataloader = DataLoader(
    ImageDataset("/content/gdrive/MyDrive/python/data/Dataset/images/%s" % dataset_name, transforms_=transforms_, unaligned=True),
    batch_size=batch_size,
    shuffle=True,
    num_workers=n_cpu,
)

# Test data loader
val_dataloader = DataLoader(
    ImageDataset("/content/gdrive/MyDrive/python/data/Dataset/images/%s" % dataset_name, transforms_=transforms_, unaligned=True, mode="test"),
    batch_size=5,
    shuffle=True,
    num_workers=1,
)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


**생성한 샘플이미지를 시각화하여 저장하는 함수를 지정**
합슥하는 동안 생성한 이미지를 확인 및 저장하기 위해 sample_images함수를 정의합니다. val_dataloader의 배치 사이즈가 5이므로 sample_images는 make_grid함수를 통해 5개씩 샘플을 생성하여 images폴더에 저장합니다.

In [ ]:
def sample_images(batches_done):
    """Saves a generated sample from the test set"""
    imgs = next(iter(val_dataloader))
    G_AB.eval()
    G_BA.eval()
    real_A = Variable(imgs["A"].type(Tensor))
    fake_B = G_AB(real_A)
    real_B = Variable(imgs["B"].type(Tensor))
    fake_A = G_BA(real_B)
    # Arange images along x-axis
    real_A = make_grid(real_A, nrow=5, normalize=True)
    real_B = make_grid(real_B, nrow=5, normalize=True)
    fake_A = make_grid(fake_A, nrow=5, normalize=True)
    fake_B = make_grid(fake_B, nrow=5, normalize=True)
    # Arange images along y-axis
    image_grid = torch.cat((real_A, fake_B, real_B, fake_A), 1)
    save_image(image_grid, "images/%s/%s.png" % (dataset_name, batches_done), normalize=False)

In [ ]:
# 모델 학습 파이프라인
prev_time = time.time()
for epoch in range(epoch_start, n_epochs):
    for i, batch in enumerate(dataloader):

        # (1) Set model input
        # dataloader에서 실제 사진 이미지 A와 애니메이션 이미지 B를 배치사이즈만큼 불러옵니다
        real_A = Variable(batch["A"].type(Tensor))
        real_B = Variable(batch["B"].type(Tensor))

        # (2) Adversarial ground truths
        # Discriminator의 레이블 값을 만듭니다. valid와 fake변수를 만듭니다. PatchGAN에 따라 valid는 Discriminator의 출력 크기만큼 전부 1로 채워지고,
        # 변수 fake는 Discriminator의 출력 크기만큼 전부 0으로 채워집니다.
        valid = Variable(Tensor(np.ones((real_A.size(0), *D_A.output_shape))), requires_grad=False)
        fake = Variable(Tensor(np.zeros((real_A.size(0), *D_A.output_shape))), requires_grad=False)

        # (3) Train Generators
        # Generator를 학습합니다. Generator G_AB와 G_BA를 학습 모드로 전환하고, optimizer_G를 zero_grad()합니다.

        G_AB.train()
        G_BA.train()

        optimizer_G.zero_grad()

        # (4) Identity loss
        # 색감, 형태 등을 유지하기 위한 Identity loss를 계산합니다. G_BA에 실제 이미지 A를 입력한 후 이를 real_A와 비교하여 L1Loss를 계산합니다.
        # 마찬가지로 G_AB에 실제 이미지 B를 입력한 후 이를 다시  real_B와 비교하여 L1Loss를 계산합니다.
        loss_id_A = criterion_identity(G_BA(real_A), real_A)
        loss_id_B = criterion_identity(G_AB(real_B), real_B)

        loss_identity = (loss_id_A + loss_id_B) / 2

        # (5) GAN loss
        # GAN Loss를 계산합니다. 이미지 real_A로부터 스타일이 변환된 가짜 이미지 fake_B를 생성합니다. Discriminator D_B는 생성된 fake_B가 진짜인지 가짜인지 분류합니다. 그리고 이 Discriminator D_B를 속이도록 Generator G_AB의 GAN Loss를 계산합니다.
        # 이미지 real_B도 마찬가지의 과정을 거쳐서 BA의 GAN Loss를 계산합니다.
        fake_B = G_AB(real_A)
        loss_GAN_AB = criterion_GAN(D_B(fake_B), valid)
        fake_A = G_BA(real_B)
        loss_GAN_BA = criterion_GAN(D_A(fake_A), valid)

        loss_GAN = (loss_GAN_AB + loss_GAN_BA) / 2

        # (6) Cycle loss
        # Cyle Loss를 계산합니다. Generator G_AB로 가짜 이미지 fake_B에서 새로운 가짜 이미지 recov_A를 생성합니다. 그리고 이를 다시 원래 실제 이미지 real_A와 비교하여 L1 Loss를 계산합니다.
        recov_A = G_BA(fake_B)
        loss_cycle_A = criterion_cycle(recov_A, real_A)
        recov_B = G_AB(fake_A)
        loss_cycle_B = criterion_cycle(recov_B, real_B)

        loss_cycle = (loss_cycle_A + loss_cycle_B) / 2

        # (7) Total loss
        # 앞서 계산한 Loss를 총합하여 Generator의 전체 손실함수를 계산하고 Generator의 가중치를 업데이트 합니다.
        loss_G = loss_GAN + lambda_cyc * loss_cycle + lambda_id * loss_identity

        loss_G.backward()
        optimizer_G.step()

        # (8) Train Discriminator A
        # Dicriminator D_A의 가중치를 업데이트 하는 과정입니다. 실제 이미지 A는 valid로 분류하고, 생성한 fake_A는 fake로 분류합니다. 학습을 위하여 optimizer D_A를 zero grad

        optimizer_D_A.zero_grad()

        # (9) Real loss
        loss_real = criterion_GAN(D_A(real_A), valid)
        # (10) Fake loss (on batch of previously generated samples)
        fake_A_ = fake_A_buffer.push_and_pop(fake_A)
        loss_fake = criterion_GAN(D_A(fake_A_.detach()), fake)
        # (11) Total loss
        loss_D_A = (loss_real + loss_fake) / 2

        loss_D_A.backward()
        optimizer_D_A.step()

        # (12) Train Discriminator B

        optimizer_D_B.zero_grad()

        # (13) Real loss
        loss_real = criterion_GAN(D_B(real_B), valid)
        # (14) Fake loss (on batch of previously generated samples)
        fake_B_ = fake_B_buffer.push_and_pop(fake_B)
        loss_fake = criterion_GAN(D_B(fake_B_.detach()), fake)
        # (15) Total loss
        loss_D_B = (loss_real + loss_fake) / 2

        loss_D_B.backward()
        optimizer_D_B.step()

        loss_D = (loss_D_A + loss_D_B) / 2

        # (16) Determine approximate time left
        batches_done = epoch * len(dataloader) + i
        batches_left = n_epochs * len(dataloader) - batches_done
        time_left = datetime.timedelta(seconds=batches_left * (time.time() - prev_time))
        prev_time = time.time()

        # (17) Print log
        sys.stdout.write(
            "\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f, adv: %f, cycle: %f, identity: %f] ETA: %s"
            % (
                epoch,
                n_epochs,
                i,
                len(dataloader),
                loss_D.item(),
                loss_G.item(),
                loss_GAN.item(),
                loss_cycle.item(),
                loss_identity.item(),
                time_left,
            )
        )

        # (18) If at sample interval save image
        if batches_done % sample_interval == 0:
            sample_images(batches_done)

    # (19) Update learning rates
    lr_scheduler_G.step()
    lr_scheduler_D_A.step()
    lr_scheduler_D_B.step()
    # (20) Save model checkpoints
    if checkpoint_interval != -1 and epoch % checkpoint_interval == 0:
        torch.save(G_AB.state_dict(), "saved_models/%s/G_AB_%d.pth" % (dataset_name, epoch))
        torch.save(G_BA.state_dict(), "saved_models/%s/G_BA_%d.pth" % (dataset_name, epoch))
        torch.save(D_A.state_dict(), "saved_models/%s/D_A_%d.pth" % (dataset_name, epoch))
        torch.save(D_B.state_dict(), "saved_models/%s/D_B_%d.pth" % (dataset_name, epoch))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[Epoch 399/400] [Batch 72/73] [D loss: 0.046236] [G loss: 1.743282, adv: 0.732759, cycle: 0.069570, identity: 0.062966] ETA: 0:00:00.322052